In [1]:
import sys
sys.path.append('/home/potzschf/repos/')
from helperToolz.helpsters import *
from helperToolz.evapo import *
import geopandas as gpd
import tarfile
from pyproj import Proj, transform
workhorse = True

if workhorse:
    origin = 'Aldhani/eoagritwin/'
else:
    origin = ''

In [2]:
#### get path and rows of scenes that have data for the chosen AOI (e.g. Brandenburg)

# load shapefiles and check projections
ger = gpd.read_file(f'/data/{origin}misc/gadm41_DEU_shp/gadm41_DEU_1.shp')
aoi = ger[ger['NAME_1'] == 'Brandenburg']

orbits = gpd.read_file(f'/data/{origin}misc/WRS2_descending_0/WRS2_descending.shp')

if aoi.crs != orbits.crs:
    orbits = orbits.to_crs(aoi.crs)

# find overlapping paths/rows
intersecting = orbits[orbits.intersects(aoi.unary_union)]
path_rows = [f'{p}_{r}' for p, r in zip(intersecting['PATH'], intersecting['ROW'])]

/tmp/ipykernel_640425/1923405511.py:13: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  intersecting = orbits[orbits.intersects(aoi.unary_union)]


In [3]:
# get all paths from downloaded products --> subsetted to paths and rows
landsat_files = getFilelist(f'/data/{origin}et/Landsat/raw/', '.tar.gz', deep=True)

# create a look-up dictionary for time subsets
lookUp = LandsatETFileManager(landsat_files)

In [10]:
#### do the compositing monthly
# subset data and extract
for landsat_file in lookUp.get_by_year_and_month(2024, 1):
    with tarfile.open(landsat_file, 'r:gz') as tar:
        tar.extractall(f'/data/{origin}et/Landsat/extracts/')
# remove not needed files
for ending in ['.xml', '.txt']:
    trash_list = getFilelist(f'/data/{origin}et/Landsat/extracts/', ending)
    for trash in trash_list:
        os.remove(trash)

/tmp/ipykernel_640425/1292985906.py:5: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(f'/data/{origin}et/Landsat/extracts/')


In [11]:
# get maximum bounding box for month and fill it with values
nc_files = getFilelist(f'/data/{origin}et/Landsat/extracts/', '.nc')


In [ ]:
for var in ds.data_vars:
    print(f"\nVariable: {var}")
    for attr, val in ds[var].attrs.items():
        print(f"  {attr}: {val}")

In [ ]:
north = ds.attrs['UpperLeftCornerLatLong'][0]
south = ds.attrs['LowerRightCornerLatLong'][0]
east = ds.attrs['LowerRightCornerLatLong'][1]
west = ds.attrs['UpperLeftCornerLatLong'][1]

In [ ]:
(ds.coords['XDim_qa_pixel'][-1] - ds.coords['XDim_qa_pixel'][0]) / 30

In [ ]:
ds.data_vars

In [ ]:
# tiff export

img = ds['ETA']
import rasterio
from rasterio.transform import from_origin

# Assuming the dataset has spatial resolution info
transform = from_origin(ds['XDim_qa_pixel'][0], ds['YDim_qa_pixel'][0], x_res, y_res)

# Define output GeoTIFF filename
output_filename = 'output_data.tif'

# Open a new GeoTIFF file for writing
with rasterio.open(output_filename, 'w', driver='GTiff', 
                   height=qa_pixel.shape[0], width=qa_pixel.shape[1],
                   count=1, dtype=qa_pixel.dtype,
                   crs="EPSG:32633", transform=transform) as dst:
    dst.write(qa_pixel, 1)

print(f"GeoTIFF saved as {output_filename}")